In [14]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skimage.feature import hog
from skimage import exposure
from sklearn.decomposition import PCA

In [39]:
def hog_features(image):
    image = cv2.resize(image, (64, 64))
    features = hog(image, pixels_per_cell=(4, 4), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
    return features

def get_data():
    features_lst, labels_lst = [], []
    im_dir = os.listdir('bbox')
    # for img_dir in im_dir:
    #     labels_lst.append('traffic-sign')
    #     img = cv2.imread(os.path.join('bbox', img_dir), cv2.IMREAD_GRAYSCALE)
    #     features_lst.append(hog_features(img))
    for img_dir in im_dir:
        number = int(img_dir.rsplit('_', 2)[-1].split('.')[0])
        labels_lst.append(number)
        img = cv2.imread(os.path.join('bbox', img_dir), cv2.IMREAD_GRAYSCALE)
        features_lst.append(hog_features(img))
    print("Read ", len(features_lst), " images")
    
    im_dir = os.listdir('bbox2')
    for img_dir in im_dir:
        labels_lst.append('background')
        img = cv2.imread(os.path.join('bbox2', img_dir), cv2.IMREAD_GRAYSCALE)
        features_lst.append(hog_features(img))
    print("Read ", len(features_lst), " images")
    return features_lst, labels_lst
# def get_data():
#     features_lst, labels_lst = [], []
#     im_dir = os.listdir('bbox')
#     for img_dir in im_dir:
#         number = int(img_dir.rsplit('_', 2)[-1].split('.')[0])
#         labels_lst.append(number)
#         img = cv2.imread(os.path.join('bbox', img_dir), cv2.IMREAD_GRAYSCALE)
#         features_lst.append(hog_features(img))
#     print("Read ", len(features_lst), " images")
#     return features_lst, labels_lst

In [40]:
features_lst, labels_lst = get_data()
classes_lst = list(set(labels_lst))

Read  14588  images
Read  30668  images


In [52]:
X_train, X_test, y_train, y_test = train_test_split(features_lst, labels_lst, test_size=0.2, random_state=42)

In [53]:
pca = PCA(n_components=500)
pca.fit(X_train)

PCA(n_components=500)

In [54]:
X_pca = pca.transform(X_train)

In [55]:
lda = LinearDiscriminantAnalysis()

# Construct a pipeline with scaling and LDA
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale features
    ('lda', lda)
])

# Train the LDA model
pipeline.fit(X_pca, y_train)

# Predict on the test set
X_test_pca = pca.transform(X_test)
y_pred = pipeline.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9572872513857189


In [45]:
from sklearn.svm import LinearSVC
svm = LinearSVC(dual=True)
svm.fit(X_pca, y_train)
X_test_pca = pca.transform(X_test)
y_pred = svm.predict(X_test_pca)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

c:\Users\ADMIN\miniconda3\envs\aphe2\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.9714704923377894


In [56]:
import joblib
joblib.dump(pca, 'pca_500.pkl')
joblib.dump(pipeline, 'lda_pca_500.pkl')
# joblib.dump(svm, 'svm_pca_500.pkl')

['lda_pca_500.pkl']

In [57]:
import joblib
pipeline_loaded = joblib.load('lda_pca_500.pkl')
pca = joblib.load('pca_500.pkl')
test_im = cv2.resize(cv2.imread('image.png', cv2.IMREAD_GRAYSCALE), (64, 64))
# test_im = cv2.resize(cv2.imread('window0_40.jpg', cv2.IMREAD_GRAYSCALE), (64, 64))
features = hog(test_im, pixels_per_cell=(4, 4), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
    
features = np.array(features).reshape(1, -1)
print(features.shape)
pca_features = pca.transform(features)
predicted_class = pipeline_loaded.predict(pca_features)
with open('data/classes.txt', 'r') as file:
    label = []
    for line in file:
        label.append(line.strip())

print("Predicted Class:", predicted_class)
if predicted_class == 'background':
    predicted_class = -1
else:
    print("Predicted Class:", label[int(predicted_class[0])])
# print(predicted_class)
# print(label[int(predicted_class[0])])


(1, 8100)
Predicted Class: ['10']
Predicted Class: No Stopping and No Parking
